##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tải dữ liệu văn bản

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Xem trên TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/vi/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Chạy trên Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/vi/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Xem mã nguồn trên GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/vi/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Tải notebook</a>
  </td>
</table>

Cộng đồng TensorFlow tại Việt Nam đã và đang dịch những tài liệu này từ nguyên bản tiếng Anh. Những bản dịch này được hoàn thiện dựa trên sự nỗ lực đóng góp từ cộng đồng lập trình viên sử dụng TensorFlow, và điều này có thể không đảm bảo được tính cập nhật của bản dịch đối với [Tài liệu chính thức bằng tiếng Anh](https://www.tensorflow.org/?hl=en) này. Nếu bạn có bất kỳ đề xuất nào nhằm cải thiện bản dịch này, vui lòng tạo Pull request đến kho chứa trên GitHub của [tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n). Để đăng ký dịch hoặc cải thiện nội dung bản dịch, các bạn hãy liên hệ và đặt vấn đề tại [docs-vi@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-vi).

Trong bài viết này chúng ta sẽ cùng tìm hiểu về cách sử dụng `tf.data.TextLineDataset` để tạo các mẫu dữ liệu từ tệp văn bản sẵn có. `TextLineDataset` được dùng để thiết kế một tập dữ liệu từ các tệp văn bản khác nhau, trong đó mỗi dòng trong văn bản gốc sẽ ứng với một mẫu trong tập dữ liệu (phù hợp với các văn bản được tổ chức theo dòng như thơ hoặc log lỗi).

Ta sẽ sử dụng ba bản dịch tiếng Anh của tác phẩm Illiad được sáng tác bởi Homer để xây dựng mô hình cho phép xuất ra bản dịch tương ứng của từng dòng trong văn bản.

## Chuẩn bị


In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

Ta sẽ sử dụng bản dịch của các dịch giả sau:

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [bản dịch](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [bản dịch](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [bản dịch](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

Các văn bản được sử dụng đều đã được tiền xử lý dữ liệu cơ bản (xóa header/footer của tài liệu, số dòng và tiêu đề chương). Chúng ta sẽ phải tải xuống các tệp này.

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

## Xây dựng tập dữ liệu từ văn bản

Đầu tiên ta sẽ tạo ra các tập dữ liệu tương ứng với từng tệp văn bản.

Về sau các tập dữ liệu này sẽ được gộp thành một tập lớn, do đó ta cần ghi chú lại tập dữ liệu gốc ứng với từng mẫu. Để làm được điều này, ta sẽ sử dụng `tf.data.Dataset.map` kết hợp với một hàm đánh nhãn. Khi truyền vào một mẫu, hàm sẽ trả về cặp dữ liệu `(example, label)` tương ứng.

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

Gộp các tập đã dán nhãn thành một tập dữ liệu lớn.

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

Để xem cặp `(example, label)` trả về, ta có thể sử dụng các hàm `tf.data.Dataset.take` and `print`. Giá trị trong thuộc tính `numpy` chính là giá trị của Tensor.

In [ ]:
for ex in all_labeled_data.take(5):
  print(ex)

## Encode mỗi dòng văn bản

Các mô hình học máy nhận đầu vào là số. Do đó để xây dựng mô hình cho dữ liệu văn bản, các chuỗi văn bản cần được chuyển thành chuỗi các số. Nói cách khác, ta cần ánh xạ mỗi từ phân biệt thành một số phân biệt. 

### Xây dựng danh sách từ vựng

Để xây dựng danh sách từ vựng, ta cần tách các chuỗi văn bản thành tập hợp các từ. Có nhiều cách để tách từ trong TensorFlow và Python. Tuy nhiên trong bài viết này, ta sẽ thực hiện như sau:

1. Xét chuỗi văn bản trong thuộc tính `numpy` của từng mẫu.
2. Sử dụng `tfds.features.text.Tokenizer` để tách chuỗi thành các token.
3. Đưa token thu được vào cấu trúc set trong Python để loại bỏ các token trùng lặp.
4. Lưu số lượng từ trong danh sách từ vựng để sử dụng về sau.

In [ ]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

### Encode các mẫu dữ liệu

Ta sẽ tạo encoder bằng cách truyền `vocabulary_set` vào `tfds.features.text.TokenTextEncoder`. Phương thức `encode` của encoder nhận đầu vào là một chuỗi văn bản và trả về là chuỗi các số.

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

Thử thực hiện encode trên một chuỗi văn bản mẫu để kiểm tra kết quả trả về.

In [ ]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

Để sử dụng encoder với toàn bộ văn bản, ta cần "đóng gói" (wrap) nó vào trong một đối tượng là `tf.py_function` và truyền đối tượng này vào phương thức `map` của tập dữ liệu.

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

Ta sẽ sử dụng `Dataset.map` để chạy hàm với từng mẫu trong tập dữ liệu. `Dataset.map` thực thi trên ở chế độ đồ thị:

* Mỗi tensor trong đồ thị không chứa giá trị.
* Ở chế độ đồ thị, ta chỉ có thể sử dụng toán tử (Ops) và hàm toán học (functions) trong TensorFlow.

Do đó ta không thể truyền trực tiếp hàm đã viết vào `.map` mà phải thông qua `tf.py_function`. `tf.py_function` sẽ truyền tensor (bao gồm giá trị và phương thức `.numpy()` để truy cập giá trị đó) vào hàm đã viết.


In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## Chia tập dữ liệu thành batch huấn luyện và batch kiểm thử

Ta sẽ sử dụng `tf.data.Dataset.take` và `tf.data.Dataset.skip` để chia tập dữ liệu thành một tập nhỏ dành cho việc kiểm thử và tập lớn hơn còn lại cho việc huấn luyện.

Trước khi đưa vào mô hình, tập dữ liệu cần được tổ chức thành từng batch. Thông thường các mẫu dữ liệu thuộc cùng một batch sẽ có cùng kích thước với nhau. Tuy nhiên trong dữ liệu hiện có, mỗi mẫu có kích thước khác nhau do mỗi dòng trong văn bản gốc có số lượng từ khác nhau. Vì thế ta sẽ sử dụng `tf.data.Dataset.padded_batch` (thay vì `batch`) để tất cả các mẫu có cùng kích thước bằng cách đệm thêm cho chúng.

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Kể từ bây giờ, dữ liệu trong `test_data` và `train_data` không tổ chức theo dạng tập hợp các cặp `(example, label)` mà là tập hợp các batch. Mỗi batch là một cặp *(nhiều mẫu, nhiều nhãn)* được biễu diễn dưới dạng mảng.

Ví dụ:

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

Số lượng từ trong danh sách từ vựng lúc này cần tăng lên 1 vì ta vừa bổ sung một token mới là số 0 được sử dụng trong quá trình đệm dữ liệu.

In [ ]:
vocab_size += 1

## Xây dựng mô hình

In [ ]:
model = tf.keras.Sequential()

Tầng đầu tiên của mô hình sẽ chuyển các chuỗi văn bản được biểu diễn dưới dạng số thành các vector embedding dày đặc. Xem thêm tại [Hướng dẫn về word embeddings](../text/word_embeddings.ipynb).

In [ ]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

Tiếp theo sẽ là tầng [Bộ nhớ Ngắn hạn Dài - *Long Short-Term Memory (LSTM)*](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) cho phép mô hình xác định ý nghĩa của một từ trong ngữ cảnh của nó. Bidirectional được bổ sung bên ngoài tầng LSTM giúp mô hình học được mối quan hệ giữa điểm dữ liệu được xét và các điểm trước và sau nó.

In [ ]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Cuối cùng ta sẽ cần một hoặc nhiều tầng kết nối dày đặc (*densely connected layer*), trong đó tầng cuối cùng sẽ đóng vai trò là tầng đầu ra. Tầng đầu ra sẽ trả về xác suất ứng với từng nhãn. Nhãn có xác suất cao nhất chính là giá trị tiên đoán của mô hình. 

In [ ]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

Ta sẽ biên dịch mô hình với hàm mất mát là `sparse_categorical_crossentropy` (dành cho mô hình phân loại) và trình tối ưu hóa là `adam` (có nhiều lựa chọn, tuy nhiên `adam` là lựa chọn phổ biến nhất).

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Huấn luyện mô hình

Mô hình đã xây dựng chạy trên tập dữ liệu cho kết quả tương đối khả quan (khoảng 83%).

In [ ]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))